In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import math 
import time
import numpy as np
from collections import Counter
from tqdm import tqdm
import array as arr
import copy

filepathTXT = "../edgelists/BlogCatalog-edgelist.txt"
filepathCSV = "../edgelists/BlogCatalog-edgelist.csv"
embeddingsrecursive = "../embeddings/BlogCatalog-edgelist.txt.embeddings-recursive"
embeddingsiterative = "../embeddings/BlogCatalog-edgelist.txt.embeddings-iterative"

In [2]:
def parseEdgeList1(graph_file, delimiter=" ", weighted=False, direction="undirected"):
    if(weighted == False):
        G = nx.read_edgelist(graph_file, delimiter=delimiter)
    else:
        G = nx.read_edgelist(graph_file, delimiter=delimiter, nodetype=int, data=(('weight',float),))
    print(G.number_of_nodes(), G.number_of_edges(), " loaded from ", graph_file)
    if(direction == "undirected"):
        return G.to_undirected()
    else:
        return G

In [3]:
def parseEdgeList2(graph_file, direction="undirected"):
    # Create Graph
    G = nx.Graph()
    # Create head
    colNames=["Start", "End"]
    edgeData = pd.read_csv(filepathCSV, names=colNames)

    #Add nodes
    nodes = []
    #loop throug data records
    for i in range (0, edgeData.shape[0]):
        #append every node
        nodes.append(edgeData.iloc[i,0])
        nodes.append(edgeData.iloc[i,1])
    #creating a set of nodes    
    nodes = set(nodes)
    #sorting the nodes in increasing order
    uniqueNodes = (list(nodes))
    uniqueNodes.sort()
    #adding the nodes to the graph
    G.add_nodes_from(uniqueNodes)

    # Add edges
    #loop from 0 to amount of records
    edgeCount = 0
    for i in range (0, edgeData.shape[0]):
        edgeCount += 1
        #add the edge to the graph
        G.add_edge(edgeData.iloc[i,0], edgeData.iloc[i,1])
    print("Nodes: ", G.number_of_nodes()," Edges: ", G.number_of_edges(), " loaded from ", graph_file)

    if(direction == "undirected"):
        return G.to_undirected()
    else:
        return G

In [4]:
# Safe graph as dict
def getAdjList(graph):
    adjdict = {}
    for neighbor in graph:
        neighbors = [n for n in G.neighbors(neighbor)]
        num_neihbors = len(neighbors)
        adjdict[neighbor] = random.sample(neighbors, num_neihbors)
    print("Created a dict with ", len(adjdict), "keys")
    return adjdict

In [5]:
def getAdjNPList(graph):
    adjdict = {}
    for vertex in graph:
        adjdict[vertex] = np.array([n for n in G.neighbors(vertex)]) 
        np.random.shuffle(adjdict[vertex])
    return adjdict


In [ ]:
femb_recursive = open(embeddingsrecursive, 'w')
femb_iterative = open(embeddingsiterative, 'w')

## Toy Graph

In [6]:
def toyGraph():
    G = nx.Graph()
    G.add_nodes_from([1,2,3,4,5,6,7,8,9,10,11])
    G.add_edge(2, 1);G.add_edge(3, 1);G.add_edge(3, 2);G.add_edge(4, 1);
    G.add_edge(4, 2);G.add_edge(4, 3);G.add_edge(5, 1);G.add_edge(6, 1);
    G.add_edge(7, 1);G.add_edge(7, 5);G.add_edge(7, 6);G.add_edge(8, 1);
    G.add_edge(8, 2);G.add_edge(8, 3);G.add_edge(8, 4);G.add_edge(9, 1); 
    G.add_edge(9, 3);G.add_edge(10, 3);G.add_edge(11, 1);G.add_edge(11, 5);
    print("Nodes: ", G.number_of_nodes()," Edges: ", G.number_of_edges())
    # Draw graph
    # nx.draw(G, with_labels = True)
    # plt.show()
    return G

## Random Walk with window

In [ ]:
def chooseNodes(list_nodes, sample_size):
    return random.sample(population=list_nodes, k=sample_size) # 2.22 s ± 179 ms per loop  (labesl str)

def getPerNodeBudget(numNodes, budget):
    return math.floor(budget/numNodes)

def storeContextPairs(context_pair, budget, context_pairs):
    if context_pair not in context_pairs:
        context_pairs[context_pair] = budget
    else:
        context_pairs[context_pair] = context_pairs[context_pair] +  budget
        
def updateContextPairs(window, context_pairs):
    lastNode = window[-1]
    labelOfLastNode, budgetOfLastNode = lastNode
    index_count = 0
    window_except_last = window[:-1]
    for node in reversed(window_except_last):
        node_label = node[0]
        if index_count == ORIGINAL_WINDOW_SIZE:
            break
        context_pair1 = str(labelOfLastNode) +","+ str(node_label)
        context_pair2 = str(node_label) + "," + str(labelOfLastNode)
        storeContextPairs(context_pair1, budgetOfLastNode, context_pairs)
        storeContextPairs(context_pair2, budgetOfLastNode, context_pairs)
        index_count = index_count + 1
    
        
def addNewNodeToWindow(window, vertex, budget):
    window.append([vertex,budget])
    return window

def BFSRandomWalkWindow(queue, context_pairs, window_size, walk_lenght):
    while queue:
        vertex, budget, current_walk_lenght, window = queue.pop(0)
        vertex_neighbors = [n for n in G.neighbors(vertex)]
        num_neighbors = len(vertex_neighbors)
        m = getPerNodeBudget(num_neighbors, budget)
        remainder = budget - (m * num_neighbors)
        chosen_nodes = []
        if remainder > 0:
            chosen_nodes = chooseNodes(vertex_neighbors, remainder)
        current_walk_lenght += 1
        for neighbor in vertex_neighbors:
            budget_for_this_node = m 
            if neighbor in chosen_nodes:
                budget_for_this_node = budget_for_this_node + 1
            if(budget_for_this_node > 0):
                if len(window) == window_size:
                    window.pop(0)
                addNewNodeToWindow(window, neighbor, budget_for_this_node)
                print("window ->", window)
                updateContextPairs(window, context_pairs) 
                print("context_pairs ->", context_pairs)
                if current_walk_lenght < walk_lenght:
                    queue.append((neighbor, budget_for_this_node, current_walk_lenght, list(window)))
                window.pop(-1)
                print(len(queue))

### testing (works)

In [ ]:
def chooseNodes(list_nodes, sample_size):
    return random.sample(population=list_nodes, k=sample_size) # 2.22 s ± 179 ms per loop  (labesl str)

def getPerNodeBudget(numNodes, budget):
    return math.floor(budget/numNodes)

def storeContextPairs(context_pair, budget, context_pairs):
    if context_pair not in context_pairs:
        context_pairs[context_pair] = budget
    else:
        context_pairs[context_pair] = context_pairs[context_pair] +  budget
        
def updateContextPairs(window, context_pairs):
    lastNode = window[-1]
    labelOfLastNode, budgetOfLastNode = lastNode
    index_count = 0
    window_except_last = window[:-1]
    for node in reversed(window_except_last):
        node_label = node[0]
        if index_count == ORIGINAL_WINDOW_SIZE:
            break
        context_pair1 = str(labelOfLastNode) +","+ str(node_label)
        context_pair2 = str(node_label) + "," + str(labelOfLastNode)
        storeContextPairs(context_pair1, budgetOfLastNode, context_pairs)
        storeContextPairs(context_pair2, budgetOfLastNode, context_pairs)
        index_count = index_count + 1
    
        
def addNewNodeToWindow(window, vertex, budget):
    window.append([vertex,budget])
    return window


# NEW
def BFSRandomWalkWindow(queue, context_pairs, window_size, walk_lenght):
    while queue:
        vertex, budget, current_walk_lenght, window = queue.pop(0)
        #print("vertex, budget, current_walk_lenght, window ->",vertex, budget, current_walk_lenght, window)
        vertex_neighbors = adjdict[vertex]
        num_neighbors = len(vertex_neighbors)
        m = getPerNodeBudget(num_neighbors, budget)
        remainder = budget - (m * num_neighbors)
        #print("remainder", remainder)

        if remainder > 0:
            np.random.shuffle(vertex_neighbors) 
        else:
            remainder = num_neighbors 
        
        #print("vertex_neighbors", vertex_neighbors)
        
        current_walk_lenght += 1

        for neighbor in vertex_neighbors[:remainder]:
            #print("chosen node from neighbours ->", neighbor)
            budget_for_this_node = m 
            
            if remainder != num_neighbors:
                budget_for_this_node = budget_for_this_node + 1

            if len(window) == window_size:
                window.pop(0)
                
            addNewNodeToWindow(window, neighbor, budget_for_this_node) 
            #print("window ->", window)
            updateContextPairs(window, context_pairs) 
            #print ("context_pairs -> ", context_pairs)
            if current_walk_lenght < walk_lenght:
                queue.append((neighbor, budget_for_this_node, current_walk_lenght, list(window)))
                #print("updated queue", queue)
            window.pop(-1)

### NP implementation

In [10]:
def chooseNodes(list_nodes, sample_size):
    return random.sample(population=list_nodes, k=sample_size) # 2.22 s ± 179 ms per loop  (labesl str)

def getPerNodeBudget(numNodes, budget):
    return math.floor(budget/numNodes)

def storeContextPairs(context_pair, budget, context_pairs):
    if context_pair not in context_pairs:
        context_pairs[context_pair] = budget
    else:
        context_pairs[context_pair] = context_pairs[context_pair] +  budget
        
def updateContextPairs(window, window_count, context_pairs):
    lastNode = window[window_count]
   
    labelOfLastNode, budgetOfLastNode = lastNode
    if window_count > ORIGINAL_WINDOW_SIZE:
        new_window = window[ORIGINAL_WINDOW_SIZE:window_count]
    else:
        new_window = window[:window_count]

    for node in new_window:
        node_label = node[0]
        context_pair1 = str(labelOfLastNode) +","+ str(node_label)
        context_pair2 = str(node_label) + "," + str(labelOfLastNode)
        storeContextPairs(context_pair1, budgetOfLastNode, context_pairs)
        storeContextPairs(context_pair2, budgetOfLastNode, context_pairs)
        #print("the context pairs: ", context_pair1, context_pair2 )
    
        
def addNewNodeToWindow(tempwindow, temp_window_count, window_size, vertex, budget):
    if temp_window_count == window_size:
        tempwindow = tempwindow[1:]
        
    newWindowElement = np.array([[vertex,budget]])
    tempwindow[temp_window_count+1] = newWindowElement
    temp_window_count+=1
    return tempwindow, temp_window_count


# NEW
def BFSRandomWalkWindow(queue, queue_count, context_pairs, window_size, walk_lenght):
   
    while queue_count > -1:
        #print("")
        #print("-----poping new element from queue----" )
        vertex, budget, current_walk_lenght, window, window_count = queue[queue_count]
        queue_count -= 1
        queue = queue[1:]
        #print("vertex, budget, current_walk_lenght, window, window_count ->",vertex, budget, current_walk_lenght, window, window_count)

        vertex_neighbors = adjdict[vertex]
        num_neighbors = vertex_neighbors.size
        m = getPerNodeBudget(num_neighbors, budget)
        remainder = budget - (m * num_neighbors)
        #print("remainder", remainder)

        if remainder > 0:
            np.random.shuffle(vertex_neighbors) 
        else:
            remainder = num_neighbors 
        
        #print("vertex_neighbors", vertex_neighbors)
        
        current_walk_lenght += 1
        
        for neighbor in vertex_neighbors[:remainder]:
            #print("")
            #print("chosen node from neighbours ->", neighbor)
            budget_for_this_node = m 
            temp_window = np.copy(window)
            temp_window_count = window_count
            
            if remainder != num_neighbors:
                budget_for_this_node = budget_for_this_node + 1
            
            #print("Budget for this node ->", budget_for_this_node)
                
            temp_window, temp_window_count = addNewNodeToWindow(temp_window, temp_window_count, window_size, neighbor, budget_for_this_node) 
            #print("updated window  ->", temp_window)

            updateContextPairs(temp_window, temp_window_count, context_pairs) 
            #print ("updated context_pairs -> ", context_pairs)
            
            if current_walk_lenght < walk_lenght:
                newQueueElement = np.array([[neighbor, budget_for_this_node, current_walk_lenght, temp_window, temp_window_count]])
                queue[queue_count+1] = newQueueElement
                queue_count+=1
                #print("updated queue", queue)

## NP Runner

In [24]:
def Runner():
    start = time.time()
    random.seed(0)
    rand=random.Random(0)
    
    context_pairs = {}    
    WINDOW_SIZE = ORIGINAL_WINDOW_SIZE*2+1
    
    print("Running BFSRandomWalkWindow...")
    count = 0
    for startvertex in adjdict.keys():
        count+=1
        if count == 10000:
            end = time.time()
            result = end - start
            print("1000 iterations in ->", result)
            break
        #print("")
        #print("Running from -> ", startvertex)
        
        window_count = 0
        WINDOW = np.zeros(shape=(WINDOW_SIZE+20,2),dtype=int)
        firstWindowElement = np.array([[startvertex, BUDGET]])
        WINDOW[window_count] = firstWindowElement
        #print("Starting window ->", WINDOW)
        
        queue_count = 0
        queue = np.zeros(shape=(BUDGET+100,5),dtype=object)
        firstQueueElement = np.array([[startvertex, BUDGET, 1, WINDOW, window_count]])
        queue[queue_count] = firstQueueElement
        #print("Starting queue ->", queue)
        
        
        BFSRandomWalkWindow(queue, queue_count, context_pairs, WINDOW_SIZE, WALK_LENGHT)

    return context_pairs


### Runner

In [20]:
def Runner():
    start = time.time()
    random.seed(0)
    rand=random.Random(0)
    
    context_pairs = {}    
    WINDOW_SIZE = ORIGINAL_WINDOW_SIZE*2+1
    WINDOW=[]
    
    print("Running BFSRandomWalkWindow...")
    count = 0
    for startvertex in adjdict.keys():
        count+=1
        if count == 100:
            end = time.time()
            result = end - start
            print("100 iterations in ->", result)
            break
        #print("")
        #print("Running from -> ", startvertex)
        WINDOW = [[startvertex, BUDGET]]
        queue = [(startvertex, BUDGET, 1, WINDOW)]
        BFSRandomWalkWindow(queue, context_pairs, WINDOW_SIZE, WALK_LENGHT)
    return context_pairs

### Parametrs

In [12]:
# Set the actual parameters and graph
WALK_LENGHT = 40
BUDGET = 80
ORIGINAL_WINDOW_SIZE = 10
#G = parseEdgeList1(filepathTXT) #String labels 

G = parseEdgeList2(filepathCSV) # Integer labels
adjdict = getAdjNPList(G)

Nodes:  10312  Edges:  333983  loaded from  ../edgelists/BlogCatalog-edgelist.csv


In [ ]:
# Set toy parameters and graph
WALK_LENGHT = 3
BUDGET = 1
ORIGINAL_WINDOW_SIZE = 1
G = toyGraph()
adjdict = getAdjNPList(G)

## Run

In [25]:
contextPairs = Runner()

Running BFSRandomWalkWindow...
1000 iterations in -> 133.92250108718872


In [ ]:
contextPairsTemple = {'8,1': 1,'1,8': 1,'4,8': 2,'8,4': 2, '1,2': 2, '2,1': 2, '6,1': 4, '1,6': 4,
 '9,3': 4, '3,9': 4, '3,8': 1, '8,3': 1, '7,5': 2, '5,7': 2, '11,1': 2, '1,11': 2, '1,7': 1,
 '7,1': 1, '2,8': 1, '8,2': 1, '3,10': 2, '10,3': 2}

if contextPairs == contextPairsTemple:
    print("Result is valid")
else:
    print("Result is NOT valid")

In [26]:
# Count the total count sum
countSum = 0
for key, value in contextPairs.items():
    countSum += value
print("Total value sums up to: ", countSum)

Total value sums up to:  29130242


In [19]:
contextPairs

{'233,1': 5,
 '1,233': 5,
 '7019,1': 2,
 '1,7019': 2,
 '5488,1': 2,
 '1,5488': 2,
 '6895,1': 3,
 '1,6895': 3,
 '394,1': 3,
 '1,394': 3,
 '8524,1': 2,
 '1,8524': 2,
 '4839,1': 8,
 '1,4839': 8,
 '8387,1': 3,
 '1,8387': 3,
 '4652,1': 7,
 '1,4652': 7,
 '7228,1': 2,
 '1,7228': 2,
 '3198,1': 10,
 '1,3198': 10,
 '5999,1': 2,
 '1,5999': 2,
 '5906,1': 5,
 '1,5906': 5,
 '6639,1': 2,
 '1,6639': 2,
 '6074,1': 1,
 '1,6074': 1,
 '9997,1': 5,
 '1,9997': 5,
 '3608,1': 5,
 '1,3608': 5,
 '4997,1': 5,
 '1,4997': 5,
 '9961,1': 4,
 '1,9961': 4,
 '4374,1': 6,
 '1,4374': 6,
 '8234,1': 4,
 '1,8234': 4,
 '867,1': 3,
 '1,867': 3,
 '2771,1': 1,
 '1,2771': 1,
 '3940,1': 1,
 '1,3940': 1,
 '7545,1': 2,
 '1,7545': 2,
 '7098,1': 8,
 '1,7098': 8,
 '4723,1': 5,
 '1,4723': 5,
 '5204,1': 4,
 '1,5204': 4,
 '5450,1': 3,
 '1,5450': 3,
 '4843,1': 2,
 '1,4843': 2,
 '753,1': 3,
 '1,753': 3,
 '5912,1': 1,
 '1,5912': 1,
 '6950,1': 5,
 '1,6950': 5,
 '3489,1': 5,
 '1,3489': 5,
 '4836,1': 2,
 '1,4836': 2,
 '2624,1': 3,
 '1,2624': 3

### Writing to file

In [ ]:
print("Writing context pairs to file...")    
#Writing to file    
for (key, value) in contextPairs.items():
    
    femb_iterative.write(str(key) + " " + str(value) + "\n" )
femb_iterative.close()

## Test

In [ ]:
# Shuffle with random
def ShuffleRandom():
    number_list = list(range(0, 1000))
    for i in range(0,10000):
        random.shuffle(number_list) 
ShuffleRandom()

In [ ]:
# Shuffle with np
def ShuffleNP():
    arr = np.arange(1000)
    for i in range (0,100000):
        np.random.shuffle(arr)
ShuffleNP()

### Loop through elements in list

In [ ]:
array  = random.sample(range(1,10000000), 1000000)
for element in array:
    for i in range(0,10):
        if element == 0:
            print("Element is zero")


### Poping elements with list

In [ ]:
array = random.sample(range(1,100000000), 100000)

In [ ]:
def popRandom(array):
    for i in range (0, len(array)):
        save = array.pop(0)
        #array.append(save) # 7s,4s,2s 
popRandom(array) 

### Poping elements with np

In [ ]:
array = np.random.randint(100000000, size=10000000)

In [ ]:
def popNP(array):
    for i in range (0, len(array)):
        element = array[0]
        #array = array[1:]
popNP(array) 

### Poping elements with list

In [ ]:
queue = [4] * 10000000

In [ ]:
for i in range (0, len(queue)):
    element = queue[0]
    #queue = queue[1:]

### update context pairs

In [ ]:

context_pairs = {}
windows = []
for j in range(0,1000000):
    window = []
    for i in range(0,10):
        array = random.sample(range(1,1000), 2)
        window.append(array)
    windows.append(window)
    
# very fast for 1mil windows
for wind in windows: 
    updateContextPairs(wind, context_pairs)   

### Writing context pairs

In [ ]:
#writing context pairs
for i in range (0, 10000000):
    #context_pair1 = str(10) +","+ str(20)
    context_pair1 = (10,20)

### value copie with np

In [ ]:
arr1 = np.array([1,2,3,4,5])
arr2 = np.array([3,4,5,3,6])
print("Before assighnment ->", arr1,arr2)
arr1 = np.copy(arr2)
print( "after assignment ->", arr1,arr2)
arr2[0] = 100
print( "after index0 assignment ->", arr1,arr2)

arr2 = np.append(arr2, 44)
print( "after append to array2 ->", arr1,arr2)



In [ ]:
a = 1
b = 2
a = b
print("after assignment", a,b)
a+=1
print("after a incrementing", a,b)

### value copie with list

In [ ]:
arr1 = [1,2,3,4,5]
arr2 = [3,4,5,3,6]
print(arr1,arr2)
arr1 = arr2
print(arr1,arr2)
arr2.append(44)
print(arr1,arr2)

### Window np append

In [ ]:
# # method
# WINDOW = np.array([(7,10)])
# toAppend = np.array([(1,3)])
# WINDOW = np.append(WINDOW, toAppend, axis = 0)
# a,b = WINDOW[0]
# print(a,b)

# 2 method
WINDOW = np.array([[7,10]])
toAppend = np.array([[1,3]])
WINDOW = np.concatenate((WINDOW, toAppend))
a,b = WINDOW[1]
print(a,b)

### .any np

In [ ]:
# array = np.random.randint(100000000, size=100000000)
# while queue.any():
#     queue = queue[1:]

## cheking lenght with len on an np array

In [ ]:
def lenfunc():
    array = np.random.randint(100000000, size=100000)
    for i in range(0,10000000):
        #lenght = len(array)
        lenght = array.size 
lenfunc()



### Adding new element with np arrays

In [ ]:
# try without appending 
queue = np.zeros(shape=(10000000,160), dtype=int)
#queue = np.array([[1, 1, 1, 1]])
newQueueElement = np.array([[4, 4, 4, 4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4]])
idx = 0
for i in range(0, 10000000):

    queue[idx] = newQueueElement
    idx +=1
    newqueue = queue[1:]
    queueCopy = np.copy(newQueueElement)
    
    #wqueue = np.append(queue, newQueueElement)
queue

### Adding new element with list 

In [ ]:
queue = [] 
#queue = []
newQueueElement = [4, 4, 4, 4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4,4, 4, 4, 4]

idx = 0
for i in range(0, 1000000):
    #queue[idx] = newQueueElement
    #idx +=1

    #newqueue = queue[1:]
    #queue.append(newQueueElement)
    ans = random.sample(population=newQueueElement, k=20)





In [ ]:
l = list(range(100000))

for i in range(0,len(l)):
    l1 = copy.copy(l)


### Global and local variables 

In [ ]:
a = 1
  
# Uses global because there is no local 'a' 
def f(): 
    print ('Inside f() : ', a )

# Variable 'a' is redefined as a local 
def g():     
    a = 2
    print ('Inside g() : ',a) 

# Uses global keyword to modify global 'a' 
def h():     
    global a 
    a = 3
    print ('Inside h() : ',a )

# Global scope 
print ('global : ',a) 
f() 
print ('global : ',a) 
g() 
print ('global : ',a) 
h() 
print ('global : ',a) 

### update context pairs

In [ ]:
window= [[2,1],[3,1],[4,1],[5,1],[],[]]
window_count = 3

window_except_last = window[:window_count]
window_except_last

In [ ]:
def updateContextPairsOriginal(window, context_pairs, original_size_window):
    lastNode = window[-1]
    labelOfLastNode, budgetOfLastNode = lastNode
    index_count = 0
    window_except_last = window[:-1]
    for node in reversed(window_except_last):
        node_label = node[0]
        if index_count == original_size_window:
            break
        context_pair1 = str(labelOfLastNode) +","+ str(node_label)
        context_pair2 = str(node_label) + "," + str(labelOfLastNode)
        print("the context pairs: ", context_pair1, context_pair2 )
    
        index_count = index_count + 1

contextPairs = {}
window= [[2,1],[3,1]]
original_size_window = 2
updateContextPairsOriginal(window, contextPairs,original_size_window)


In [ ]:
def updateContextPairsNew(window, window_count, context_pairs ,original_size_window):
    lastNode = window[window_count]
   
    labelOfLastNode, budgetOfLastNode = lastNode
    if window_count > original_size_window:
        new_window = window[original_size_window:window_count]
        print("new window ->", new_window)
    else:
        new_window = window[:window_count]

    for node in new_window:
        node_label = node[0]
        context_pair1 = str(labelOfLastNode) +","+ str(node_label)
        context_pair2 = str(node_label) + "," + str(labelOfLastNode)
        storeContextPairs(context_pair1, budgetOfLastNode, context_pairs)
        storeContextPairs(context_pair2, budgetOfLastNode, context_pairs)
        print("the context pairs: ", context_pair1, context_pair2 )
        
    

contextPairs = {}
# window = [[2,1],[3,1],[]]
window = [[2,1],[3,1],[]]
#window = [[2,1],[3,1],[4,1],[5,1],[]]
#window = [[2,1],[3,1],[4,1],[5,1],[6,1],[]]
window_count = 1
original_size_window = 2
updateContextPairsNew(window, window_count ,contextPairs, original_size_window)
